# Table of contents
* [Categorical Variables](#1)
* [Numerical Features (subset)](#2)
* [Absence vs Features](#3)
* [Map](#4)
* [Fit Model for Absence](#5)
* [Model Performance](#6)

In [ ]:
# packages

# standard
import numpy as np
import pandas as pd
import time

# plots
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns

# map
import folium

# ML
import h2o
from h2o.estimators import H2ORandomForestEstimator

# show all columns
pd.options.display.max_columns = None

In [ ]:
# file overview
!ls -l ../input/seagrass-and-hydrographic-data/

In [ ]:
# load data
df_0 = pd.read_csv('../input/seagrass-and-hydrographic-data/absence.txt', sep='\t')
df_1 = pd.read_csv('../input/seagrass-and-hydrographic-data/presence.txt', sep='\t')

# and combine two tables into one data frame
df = pd.concat([df_0,df_1])
df.head()

In [ ]:
# overview
df.info(verbose=True, show_counts=True) # show all columns

<a id='1'></a>
# Categorical Variables

In [ ]:
# evaluate BIO_CLASS
df.BIO_CLASS = df.BIO_CLASS.astype('category')
print(df.BIO_CLASS.value_counts())
# plot frequencies
df.BIO_CLASS.value_counts().plot(kind='bar', figsize=(10,4))
plt.title('BIO_CLASS')
plt.grid()
plt.show()

#### Very unbalanced... Let's create a binary version of that:

In [ ]:
# binary version of class => use as target for model later
df['ABSENCE'] = np.where(df.BIO_CLASS == 'absence', 1, 0)
print(df.ABSENCE.value_counts())
# plot frequencies
df.ABSENCE.value_counts().plot(kind='bar', figsize=(10,4))
plt.title('ABSENCE')
plt.grid()
plt.show()

In [ ]:
# evaluate Substrate
df.Substrate = df.Substrate.astype('category')
print(df.Substrate.value_counts())
# plot frequencies
df.Substrate.value_counts().plot(kind='bar', figsize=(10,4))
plt.title('Substrate')
plt.grid()
plt.show()

<a id='2'></a>
# Numerical Features (subset)

In [ ]:
# select subset of numerical features
features_num_select = ['LONGITUDE', 'LATITUDE', 
                       'VOTEMPER_2015_winter', 'VOTEMPER_2015_spring',
                       'VOTEMPER_2015_summer', 'VOTEMPER_2015_autumn',
                       'VOTEMPER_2015_year',
                       'maxTemp_year', 'minTemp_year', 
                       'VOTEMPER_2015_winter_maxDepth', 'VOTEMPER_2015_spring_maxDepth',
                       'VOTEMPER_2015_summer_maxDepth', 'VOTEMPER_2015_autumn_maxDepth', 
                       'VOTEMPER_2015_year_maxDepth', 
                       'maxTemp_year_maxDepth', 'minTemp_year_maxDepth',
                       'VOSALINE_2015_winter', 'VOSALINE_2015_spring',
                       'VOSALINE_2015_summer', 'VOSALINE_2015_autumn',
                       'VOSALINE_2015_year','maxVosa_year', 'minVosa_year',
                       'VOSALINE_2015_winter_maxDepth', 'VOSALINE_2015_spring_maxDepth',
                       'VOSALINE_2015_summer_maxDepth', 'VOSALINE_2015_autumn_maxDepth',
                       'VOSALINE_2015_year_maxDepth', 
                       'maxVosa_year_maxDepth', 'minVosa_year_maxDepth',
                       'CHL_2015_winter', 'CHL_2015_spring',
                       'CHL_2015_summer', 'CHL_2015_autumn',
                       'CHL_2015_year',
                       'maxCHL_year', 'minCHL_year',
                       'NIT_2015_winter', 'NIT_2015_spring',
                       'NIT_2015_summer', 'NIT_2015_autumn',
                       'NIT_2015_year', 'maxNIT_year', 'minNIT_year',
                       'NIT_2015_winter_maxDepth', 'NIT_2015_spring_maxDepth',
                       'NIT_2015_summer_maxDepth', 'NIT_2015_autumn_maxDepth',
                       'NIT_2015_year_maxDepth',
                       'maxNIT_year_maxDepth', 'minNIT_year_maxDepth',
                       'ZSD_2015_winter', 'ZSD_2015_spring',
                       'ZSD_2015_summer', 'ZSD_2015_autumn',
                       'ZSD_2015_year',
                       'maxZSD_year', 'minZSD_year',
                       'PHO_2015_winter', 'PHO_2015_spring',
                       'PHO_2015_summer', 'PHO_2015_autumn',
                       'PHO_2015_year',
                       'maxPHO_year', 'minPHO_year',
                       'PHO_2015_winter_maxDepth', 'PHO_2015_spring_maxDepth',
                       'PHO_2015_summer_maxDepth', 'PHO_2015_autumn_maxDepth',
                       'PHO_2015_year_maxDepth',
                       'maxPHO_year_maxDepth', 'minPHO_year_maxDepth',
                       'VHM0_2015_winter', 'VHM0_2015_spring',
                       'VHM0_2015_summer', 'VHM0_2015_autumn', 
                       'VHM0_2015_year',
                       'maxVHM0_year', 'minVHM0_year',
                       'Med_bathym', 
                       'Distance_to_Major_Cities', 'Distance_to_Complete_Cities',
                       'Distance_to_Port', 'Distance_to_Major_River',
                       'Distance_to_Complete_River',
                       'Distance_to_Coast']

In [ ]:
len(features_num_select)

In [ ]:
fig, axs = plt.subplots(22, 4, figsize=(16,100))
i = 0
for f in features_num_select:
    current_ax = axs.flat[i]
    current_ax.hist(df[f], bins=50)
    current_ax.set_title(f)
    current_ax.grid()
    i = i + 1

<a id='3'></a>
# Absence vs Features

### Impact of Substrate

In [ ]:
# cross table - absolute counts...
ctab = pd.crosstab(df.ABSENCE, df.Substrate)
ctab

In [ ]:
# ...and normalized by column
ctab_norm = ctab / ctab.sum()

plt.figure(figsize=(12,4))
sns.heatmap(ctab_norm, cmap='Blues',
            annot=True,
            vmin=0, vmax=1,
            linecolor='black',
            linewidths=0.1)
plt.show()

### Impact of Numerical Features

In [ ]:
df.ABSENCE = df.ABSENCE.astype('category')

In [ ]:
fig, axs = plt.subplots(22, 4, figsize=(16,120))
i = 0
for f in features_num_select:
    current_ax = axs.flat[i]
    sns.violinplot(ax=current_ax, x=f, y='ABSENCE', data=df)
    current_ax.set_title(f)
    current_ax.grid()
    i = i + 1

<a id='4'></a>
# Map

In [ ]:
# aux column, bio_class as integer
df['COLOR'] = df.BIO_CLASS.cat.codes

In [ ]:
# interactive map showing bio class
zoom_factor = 5 # inital map size

my_map_1 = folium.Map(location=[38,15], zoom_start=zoom_factor)

v_min = np.min(df.COLOR)
v_max = np.max(df.COLOR)

for i in range(0,df.shape[0]):
    v = df.iloc[i]['COLOR']
    v_norm = (v-v_min)/(v_max-v_min)
    i_col = int(v_norm*256)
    current_color = matplotlib.colors.to_hex(matplotlib.cm.rainbow(i_col))
    folium.CircleMarker(
       location=[df.iloc[i]['LATITUDE'], df.iloc[i]['LONGITUDE']],
       radius=6,
       popup='BIO_CLASS: ' + df.iloc[i]['BIO_CLASS'],
       color='black',
       weigth=1, # stroke width in pixel
       opacity=0.25, # stroke opacity
       fill=True,
       fill_color=current_color,
       fill_opacity=0.25
    ).add_to(my_map_1)

my_map_1 # display

In [ ]:
# interactive map showing absence / non-absence only
zoom_factor = 5 # inital map size

my_map_2 = folium.Map(location=[38,15], zoom_start=zoom_factor)

for i in range(0,df.shape[0]):
    v = df.iloc[i]['ABSENCE']    
    if (v==0):
        current_color = 'red'
    else:
        current_color = 'grey'
        
    folium.CircleMarker(
       location=[df.iloc[i]['LATITUDE'], df.iloc[i]['LONGITUDE']],
       radius=6,
       popup='BIO_CLASS: ' + df.iloc[i]['BIO_CLASS'],
       color='black',
       weigth=1, # stroke width in pixel
       opacity=0.25, # stroke opacity
       fill=True,
       fill_color=current_color,
       fill_opacity=0.25
    ).add_to(my_map_2)

my_map_2 # display

<a id='5'></a>
# Fit Model for Absence

In [ ]:
# select predictors
predictors = features_num_select + ['Substrate']
print('Number of predictors: ', len(predictors))
print(predictors)

In [ ]:
# start H2O
h2o.init(max_mem_size='12G', nthreads=4) # Use maximum of 12 GB RAM and 4 cores

In [ ]:
# upload data frame in H2O environment
t1 = time.time()
df_hex = h2o.H2OFrame(df)
t2 = time.time()
print('Elapsed time [s]: ', np.round(t2-t1,2))

In [ ]:
# define target
target = 'ABSENCE'
# explicitly convert target to categorical => classification problem
df_hex[target] = df_hex[target].asfactor()

In [ ]:
# train / test split
train_perc = 0.7
train_hex, test_hex = df_hex.split_frame(ratios=[train_perc], seed=999)

#### Check distribution of target in train / test

In [ ]:
print('Target distribution on training data:')
print(train_hex[target].as_data_frame().value_counts())
print()
print('Target distribution on test set:')
print(test_hex[target].as_data_frame().value_counts())

In [ ]:
# define (distributed) random forest model
fit_DRF = H2ORandomForestEstimator(ntrees=40,
                                   max_depth=20,
                                   min_rows=10,
                                   nfolds=5,                                   
                                   seed=999)

# train model
t1 = time.time()
fit_DRF.train(x=predictors,
              y=target,
              training_frame=train_hex)
t2 = time.time()
print('Elapsed time [s]: ', np.round(t2-t1,2))

In [ ]:
# show training scoring history
fit_DRF.plot()

In [ ]:
# variable importance
fit_DRF.varimp_plot(25)

In [ ]:
# alternative variable importance using SHAP
t1 = time.time()
fit_DRF.shap_summary_plot(train_hex);
t2 = time.time()
print('Elapsed time [s]: ', np.round(t2-t1,2))

In [ ]:
# cross validation metrics
fit_DRF.cross_validation_metrics_summary()

In [ ]:
# ROC Curve on cross validations
perf_cv = fit_DRF.model_performance(xval=True)
perf_cv.plot()

<a id='6'></a>
# Model Performance

### Performance on training set

In [ ]:
pred_train = fit_DRF.predict(train_hex)
# add actual target
pred_train['target'] = train_hex[target]
pred_train = pred_train.as_data_frame()
pred_train.head()

In [ ]:
# plot probabilities
plt.figure(figsize=(8,4))
plt.hist(pred_train.p1, bins=30)
plt.title('Predictions on Training Data')
plt.grid()
plt.show()

In [ ]:
# confusion matrix; rows ~ actual observations, cols ~ predictions
conf_train = pd.crosstab(pred_train['target'], pred_train['predict'])
# visualize
sns.heatmap(conf_train, cmap='Blues', annot=True, 
            cbar=False, fmt='d',
            linecolor='black',
            linewidths=0.1)
plt.show()

### Performance on Test Set

In [ ]:
# predict
pred_test = fit_DRF.predict(test_hex)
# add actual target
pred_test['target'] = test_hex[target]
pred_test = pred_test.as_data_frame()
pred_test.head()

In [ ]:
# plot probabilities
plt.figure(figsize=(8,4))
plt.hist(pred_test.p1, bins=30)
plt.title('Predictions on Test Set')
plt.grid()
plt.show()

In [ ]:
# confusion matrix; rows ~ actual observations, cols ~ predictions
conf_test = pd.crosstab(pred_test['target'], pred_test['predict'])
# visualize
sns.heatmap(conf_test, cmap='Blues', annot=True, 
            cbar=False, fmt='d',
            linecolor='black',
            linewidths=0.1)
plt.show()